<a href="https://colab.research.google.com/github/LiamCarter2000/LiamCarter2000/blob/main/3%2B3_Simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv("Decision Grid.csv", usecols=[1, 2, 3, 4, 5, 6, 7, 8])

df.columns = ['Treated', 'DLT', 'Pending', 'State', 'Orig_33_Action', 'IQ_33_Action', 'Orig_6_Action', 'IQ_6_Action']

decision_dict_IQ_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_33_Action'].to_dict()
decision_dict_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_33_Action'].to_dict()
decision_dict_IQ_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_6_Action'].to_dict()
decision_dict_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_6_Action'].to_dict()

In [3]:
print(df.head(30))

    Treated  DLT  Pending   State Orig_33_Action IQ_33_Action Orig_6_Action  \
0         0    0        0    open           Stay         Stay          Stay   
1         0    0        0  closed           Stay         Stay          Stay   
2         1    0        0    open           Stay         Stay          Stay   
3         1    0        0  closed           Stay         Stay          Stay   
4         1    1        0    open           Stay         Stay          Stay   
5         1    1        0  closed           Stay         Stay          Stay   
6         1    0        1    open           Stay         Stay          Stay   
7         1    0        1  closed           Stay         Stay          Stay   
8         2    0        0    open           Stay         Stay          Stay   
9         2    0        0  closed           Stay         Stay          Stay   
10        2    1        0    open           Stay         Stay          Stay   
11        2    1        0  closed           Stay    

In [4]:
print(f"Decision for Row index 29 3+3 (3,0,1,open): {decision_dict_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ 3+3 (3,0,1,open): {decision_dict_IQ_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 rolling 6 (3,0,1,open): {decision_dict_IQ_6.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ rolling 6 (3,0,1,open): {decision_dict_6.get((3, 0, 1, 'open'))}")


Decision for Row index 29 3+3 (3,0,1,open): Suspend
Decision for Row index 29 IQ 3+3 (3,0,1,open): Stay
Decision for Row index 29 rolling 6 (3,0,1,open): Stay
Decision for Row index 29 IQ rolling 6 (3,0,1,open): Stay


Data is processed into four python dictionaries: one for each of the methodologies for which the simulation will run (original 3+3, IQ 3+3, original rolling 6, IQ rolling 6). The values in each of the columns of these methodologies are as follows.

 Defines what dose level the next patient should be treated at, based on the current number of patients being treated, the number of reported DLTs, and the number of results pending at the current dose level.

Possible Dispositions:

1. Up          = start testing patients at the next higher dose level
2. Down     = start testing patients at the next lower dose level
3. Stay       = keep testing patients at the current dose level
4. Suspend = wait for additional testing to finish at current dose
                      level, before starting a test on another patient.
5. MTD       = current dose is the Maximum Tolerated Dose (MTD)


The value of having this as a python dictionary is that all values can be looked up in place for a much faster run time. So if we want to know what to do for IQ 3+3 when we have Treated = 3, DLT = 0, Pending = 1, and State = open, we can just query the corresponding dictionary with "decision_dict_IQ_33.get((3, 0, 1, 'open'))" to get "Stay"

All that is left is to simulate the actions of the dispositions while also considering possible dropout of people that develop DLT.




TODO:
* Not sure what to do if reach error state in graph. Perhaps an algorithm to return to closest valid state on decision_dict

In [5]:
Starting_Dose_Level = 2
Lowest_Possible_Dose_Level = 1
Highest_Possible_Dose_Level = 5

#(maximum duration interested patients will wait for a slot)
Waitlist_Time = 0
#(duration of DLT evaluation period)
Course_Length = 28
#(percent who will fail the screening process)
Screen_Failure_Prob = 0.30
#(percent inevaluable.  Actual inevaluable rate will be lower due to competing DLT risk)
Inevaluable_Prob = 0.20

#DLT Probability Function (defines the probability of a patient having a DLT event, based on dose)
def get_dlt_probability(dose_level):
    """Using values from the paper"""
    # Formula: 100 * (0.5 + atan(0.2 * pi * (curr_dose_level - 8.5)) / pi)
    val = 0.5 + (math.atan(0.2 * math.pi * (dose_level - 8.5)) / math.pi)
    return max(0.0, min(1.0, val))

#Patient Inter-Arrival Time (days between arrivals)
def sample_interarrival():
    return math.floor(np.random.exponential(10))  # Mean of 10 days

#Screening Duration Distribution (length of screening process)
def sample_screening_duration():
    # beta(0, 28, 1, 1) is a uniform distribution between 0 and 28
    return math.floor(np.random.uniform(0, 28))

#Time Until DLT Event (distribution of times before the patient will have the DLT event)
def sample_dlt_timing():
    # beta(0, CourseLength, 1.5, 1) skewed toward the end of the cycle
    return math.floor(np.random.beta(1.5, 1) * Course_Length)

#Time Until Inevaluable (distribution of times before the patient will become inevaluable)
def sample_inevaluable_timing():
    # beta(0,CourseLength,1,1) is a Uniform(0, 28) distribution
    return math.floor(np.random.beta(1, 1) * Course_Length)



Now to make a class for the average patient

In [6]:
class Patient:
  def __init__(self, arrival_time, dose, course_length = Course_Length):
    self.arrival_time = arrival_time
    self.dose = dose #int 1-5 inclusive
    self.course_length = course_length #length of study
    self.started_treatment = False
    self.treatment_start_time = None

    self.screening_outcome, self.days_to_screening_outcome = self.determine_screening_outcome()

    self.started_screening = False
    self.screening_start_time = None
    self.screening_end_time = None

    self.treatment_outcome, self.days_to_treatment_outcome = self.determine_patient_outcome()

    self.started_treatment = False
    self.treatment_start_time = None
    self.resolution_time = None


  def determine_screening_outcome(self):
    if np.random.random() < Screen_Failure_Prob:
      return "fail", 0
    else:
      return "pass", sample_screening_duration()

  def start_screening(self, current_clock_time):
    self.started_screening = True
    self.screening_start_time = current_clock_time
    self.screening_end_time = self.screening_start_time + self.days_to_screening_outcome


  # returns "dlt", "ineval", or "pass", as well as a duration til it happens
  def determine_patient_outcome(self):
    dlt_prob = get_dlt_probability(self.dose)
    ineval_prob = Inevaluable_Prob

    if np.random.random() < dlt_prob:
      time_to_dlt = sample_dlt_timing()
      return "dlt", time_to_dlt
    elif np.random.random() < ineval_prob:
      time_to_ineval = sample_inevaluable_timing()
      return "ineval", time_to_ineval
    else:
      return "pass", self.course_length

  # current_clock_time will be an int representation of days since trial
  # resolution_time is the day they pass, get dlt, or become ineval
  def start_treatment(self, current_clock_time):
    self.started_treatment = True
    self.treatment_start_time = current_clock_time
    self.resolution_time = self.treatment_start_time + self.days_to_treatment_outcome


In [23]:
def archive(treated_list, pending_list):
  treated_list.extend(pending_list)
  pending_list.clear()
  # treated_list.extend(screening_list)
  # screening_list.clear()


In [21]:
from typing_extensions import final
def simulation(trial_type):
  if trial_type == 1:
    decision_dict = decision_dict_33
  elif trial_type == 2:
    decision_dict = decision_dict_IQ_33
  elif trial_type == 3:
    decision_dict = decision_dict_6
  elif trial_type == 4:
    decision_dict = decision_dict_IQ_6

  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  days_before_next_patient = sample_interarrival()

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}
  total_inevals = 0

  # log
  state_log = []

  while trial_status and current_clock_time < 2000:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients
    # without queue, turned away if not needed that day
    if days_before_next_patient <= 0 and patient_needed:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level)
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1

    # checking screening list (patient queued if not needed, no need to waste a screened patient, dose level subject to change if Up or Down)
    # no need for a loop if we just want to add 1 at most on a given day
    if screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
      elif patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        patient_states["pending"] += 1
        patient_states["treated"] += 1


    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        if patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
          treated_list.append(patient)
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        elif patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Eror")

    if current_trial_event == "Stay":
      patient_needed = True
    elif current_trial_event == "Suspend":
      patient_needed = False
    elif current_trial_event == "Up":
      archive(treated_list, pending_list)

      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True

      new_state = "closed" if dose_level == max_dose_level else "open"
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})

      print(f"dose escalated to : {dose_level}")
    elif current_trial_event == "Down":
      archive(treated_list, pending_list)

      dose_level -= 1
      patient_needed = True

      # this dose level is now considered toxic
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})

      print(f"Dose de-escalated to : {dose_level}")
    elif current_trial_event == "MTD":
      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

      print(f"MTD Reached on day {current_clock_time}, with dose level {final_dose_level}")
    elif current_trial_event == "Error":
      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False


    snapshot = patient_states.copy()
    snapshot['day'] = current_clock_time
    snapshot['dose'] = dose_level
    snapshot['action'] = current_trial_event
    state_log.append(snapshot)

    current_clock_time += 1


  print(f"Final Dose Level: {final_dose_level}")
  print(f"Final Patient States: {patient_states}")
  print(f"Total Time Taken: {current_clock_time} days or ~{current_clock_time // 30} months")
  print(f"Overlooked patients: {overlooked_patients}")

  return state_log, end_on_error


In [ ]:
state_log, e = simulation(4)

dose escalated to : 3
dose escalated to : 4
Dose de-escalated to : 3
MTD Reached on day 549, with dose level 3
Final Dose Level: 3
Final Patient States: {'treated': 5, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'closed'}
Total Time Taken: 550 days or ~18 months
Overlooked patients: 33


In [ ]:
df_log = pd.DataFrame(state_log)
df_log

,treated,DLT,pending,AboveLevelState,day,dose,action
0,0,0,0,open,1,2,Stay
1,0,0,0,open,2,2,Stay
2,0,0,0,open,3,2,Stay
3,0,0,0,open,4,2,Stay
4,0,0,0,open,5,2,Stay
...,...,...,...,...,...,...,...
725,6,0,2,closed,726,5,Suspend
726,6,0,2,closed,727,5,Suspend
727,6,0,2,closed,728,5,Suspend
728,6,0,2,closed,729,5,Suspend


In [24]:
debug_chart = None
for i in range(10):
  print(f"Trial {i}")
  chart, end_on_error = simulation(2)
  if end_on_error:
    debug_chart = pd.DataFrame(chart)
    break
  print("-----------\n-----------")


Trial 0
Dose de-escalated to : 1
MTD Reached on day 210, with dose level 1
Final Dose Level: 1
Final Patient States: {'treated': 7, 'DLT': 0, 'pending': 2, 'AboveLevelState': 'closed'}
Total Time Taken: 211 days or ~7 months
Overlooked patients: 3
-----------
-----------
Trial 1
dose escalated to : 3
dose escalated to : 4
dose escalated to : 5
MTD Reached on day 605, with dose level 5
Final Dose Level: 5
Final Patient States: {'treated': 8, 'DLT': 1, 'pending': 2, 'AboveLevelState': 'closed'}
Total Time Taken: 606 days or ~20 months
Overlooked patients: 13
-----------
-----------
Trial 2
dose escalated to : 3
dose escalated to : 4
dose escalated to : 5
MTD Reached on day 371, with dose level 5
Final Dose Level: 5
Final Patient States: {'treated': 5, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'closed'}
Total Time Taken: 372 days or ~12 months
Overlooked patients: 1
-----------
-----------
Trial 3
dose escalated to : 3
dose escalated to : 4
dose escalated to : 5
MTD Reached on day 532, w

In [16]:
debug_chart